In [161]:
using Zygote, ReverseDiff, ForwardDiff, LinearAlgebra
using BenchmarkTools

In [91]:
"""
u = Ax + some constant col vector
"""
function f_test_1(A, x)
    u = A*x[2:end] .+ x[1]
    return u
end

"""
alloc inside
"""
function f_test_2(A, x)
    u = Vector{Float64}(undef, length(x)-1)
    u .= A*x[2:end] .+ x[1]
    return u
end

"""
non-alloc ver
"""
function f_test_3!(u, A, x)
    u .= A*x[2:end] .+ x[1]
end


"""
unrolled loop ver (nonalloc)
"""
function f_test_4!(u, A, x)
    row_idx = 1:size(A)[1]; col_idx = 1:size(A)[2]
    for j ∈ col_idx
        for i ∈ row_idx
            u[i] += A[i,j]*x[j+1]
        end
    end
    for i ∈ row_idx
        u[i] += x[1]
    end
end

"""
= _4 but allocates & returns the output explicitly
"""
function f_test_4b(A, x)
    row_size = size(A)[1]
    row_idx = 1:row_size; col_idx = 1:size(A)[2]
    u = Vector{Float64}(undef, row_size); fill!(u, 0.)
    for j ∈ col_idx
        for i ∈ row_idx
            u[i] += A[i,j]*x[j+1]
        end
    end
    for i ∈ row_idx
        u[i] += x[1]
    end
    return u
end

# derivative examples:
J_f_1(A, x) = Zygote.jacobian(x -> f_test_1(A, x), x)
J_f_2(A, x) = Zygote.jacobian(x -> f_test_2(A, x), x)
J_f_3(u, A, x) = Zygote.jacobian(x -> f_test_3!(u, A, x), x)

J_f_3 (generic function with 1 method)

In [95]:
# test feval:
A = Matrix{Float64}(LinearAlgebra.I, 5, 5)
u = Vector{Float64}(undef, 5); fill!(u, 0.)
x = ones(6)
f_test_3!(u, A, x)
display(u)

fill!(u, 0.)
f_test_4!(u, A, x)
display(u)

5-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0
 2.0

5-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0
 2.0

In [86]:
# test FD:
A = Matrix{Real}(LinearAlgebra.I, 5, 5) 
u = Vector{Real}(undef, 5); fill!(u, 0.)
ForwardDiff.jacobian(x -> f_test_3!(u, A, x), x)
# only works when arrays' datatype = Real (which allocates to memoryu ⟹ slow)

5×6 Matrix{Real}:
 1.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0

In [87]:
# test RD:
A = Matrix{Float64}(LinearAlgebra.I, 5, 5) 
u = Vector{Float64}(undef, 5); fill!(u, 0.)
#Zygote.jacobian(x -> f_test_3!(u, A, x), x)
Zygote.jacobian(x -> f_test_1(A, x), x)
# only works for f_test_1

([1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 1.0 0.0; 1.0 0.0 … 0.0 1.0],)

In [ ]:
# test [RD FD] on [_4 _4b]:
fill!(u, 0.)
#ForwardDiff.jacobian(x -> f_test_4b(A, x), x)
Zygote.jacobian(x -> f_test_4!(u, A, x), x)
#=
_4:
    - FD: output not detected...
    - RD: idem
_4b:
    - FD: asks the output to be Real
    - RD: usual 'mutating array' error
=#

In [2]:
# closure for tapes by @Steffen:
function comp(A)
    function f(x) 
        return A*x[2:end] .+ x[1] 
    end
    return f 
end

A = Matrix{Float64}(LinearAlgebra.I, 5,5)
x = ones(6)

f_tape = ReverseDiff.JacobianTape( comp(A), x)
comp_f_tape = ReverseDiff.compile(f_tape)

res = Array{Float64}(undef, (size(A)[1], length(x)))

ReverseDiff.jacobian!(res, comp_f_tape, x)

5×6 Matrix{Float64}:
 1.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0

### Taping experiments:

In [170]:
# functions to be tested:
f(x, y) = sum(x.*y)
function g(x, Y, n_row, n_col)
    return map(i -> f(x, @view Y[:, i]), 1:n_col)
end
function g_closure(x) # so it can be differentiatied wrt x only
    return g(x, Y, n_row, n_col)
end

g_closure (generic function with 1 method)

In [171]:
# gradient functions:
function no_tape(x, Y, n_row, n_col)
    out = Matrix{Float64}(undef, n_row, n_col)
    for i ∈ 1:n_col
        out[:, i] = ReverseDiff.gradient(x -> f(x, (@view Y[:, i])), x)
    end
    return out
end

function taped(x, Y, n_row, n_col)
    g_tape = ReverseDiff.JacobianTape(g_closure, (x))
    compiled_g_tape = ReverseDiff.compile(g_tape)
    inputs = x
    results = Matrix{Float64}(undef, n_col, n_row)
    ReverseDiff.jacobian!(results, compiled_g_tape, inputs)
    return transpose(results)
end

taped (generic function with 3 methods)

In [172]:
# initialize data:
n_row, n_col = (5, 100)
x = ones(n_row)
Y = rand(n_row, n_col)
display(Y)
#=
to test the correctness of the functions:
g(x, Y, n_row, n_col)
g_closure(x)
=#

5×100 Matrix{Float64}:
 0.0902699  0.180404  0.273062  0.842055   …  0.123996   0.70717   0.486043
 0.355467   0.781166  0.231785  0.0948917     0.925708   0.279357  0.973246
 0.723439   0.214456  0.573253  0.434728      0.0597611  0.686578  0.579483
 0.691432   0.800322  0.372804  0.160001      0.941417   0.992057  0.366285
 0.123431   0.272564  0.152883  0.104103      0.455321   0.7161    0.94599

In [173]:
no_tape(x, Y, n_row, n_col)

5×100 Matrix{Float64}:
 0.0902699  0.180404  0.273062  0.842055   …  0.123996   0.70717   0.486043
 0.355467   0.781166  0.231785  0.0948917     0.925708   0.279357  0.973246
 0.723439   0.214456  0.573253  0.434728      0.0597611  0.686578  0.579483
 0.691432   0.800322  0.372804  0.160001      0.941417   0.992057  0.366285
 0.123431   0.272564  0.152883  0.104103      0.455321   0.7161    0.94599

In [174]:
taped(x, Y, n_row, n_col)

5×100 transpose(::Matrix{Float64}) with eltype Float64:
 0.0902699  0.180404  0.273062  0.842055   …  0.123996   0.70717   0.486043
 0.355467   0.781166  0.231785  0.0948917     0.925708   0.279357  0.973246
 0.723439   0.214456  0.573253  0.434728      0.0597611  0.686578  0.579483
 0.691432   0.800322  0.372804  0.160001      0.941417   0.992057  0.366285
 0.123431   0.272564  0.152883  0.104103      0.455321   0.7161    0.94599

In [175]:
@benchmark no_tape($x, $Y, $n_row, $n_col)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  42.900 μs …   6.827 ms  ┊ GC (min … max):  0.00% … 98.80%
 Time  (median):     45.600 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   58.756 μs ± 234.927 μs  ┊ GC (mean ± σ):  18.13% ±  4.52%

   ▃▇█▇▆▅▄▄▃▂▂▁▂▁                                              ▂
  █████████████████▇▇▇▆▆▇▇▆▅▇▅▆▆▆▆▆▆▇███████▇█▇▇▆▆▆▇▇▆▄▆▅▆▅▄▅▄ █
  42.9 μs       Histogram: log(frequency) by time      80.7 μs <

 Memory estimate: 102.50 KiB, allocs estimate: 1601.

In [176]:
@benchmark taped($x, $Y, $n_row, $n_col)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  334.700 μs …   7.593 ms  ┊ GC (min … max): 0.00% … 94.22%
 Time  (median):     350.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   371.120 μs ± 271.316 μs  ┊ GC (mean ± σ):  3.15% ±  4.10%

    ▆█▅▂                                                         
  ▃▆████▇▆▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂ ▃
  335 μs           Histogram: frequency by time          519 μs <

 Memory estimate: 102.23 KiB, allocs estimate: 1714.